#### [Cypher](https://neo4j.com/product/cypher-graph-query-language/)

Cypher is a graph-optimized query language that understands, and takes advantage of, data connections. It follows connections – in any direction – to reveal previously unknown relationships and clusters. Cypher queries are much easier to write than massive SQL joins. Compare this Cypher query to its equivalent in SQL.

Neo4j and Cypher Under the Hood

Cypher is an expressive language with advanced graph patterns and collection support. Under the hood, the cypher processing pipeline first parses the query if not in cache, then goes through semantic verification and rewriting of the AST, followed by finding the cheapest execution plan (logical and physical) for all the operations using available planners, all the way to query execution.


In [1]:
import json, re
from nltk import word_tokenize

from pygments.lexers import get_lexer_by_name
from cypher_parser import CyqueryStatmentParser
lexer = get_lexer_by_name("py2neo.cypher")

alias_pattern = re.compile(r'(t[1-9]|[a-z])')	
labels_pattern = re.compile(r':`[a-z|A-Z].*`')

from process_cypher import CLAUSE_KEYWORDS, CYPHER_OPERATORS, WHERE_OPS, UNIT_OPS, AGG_OPS, ORDER_OPS, TABLE_TYPE, DerivedFieldAliasError, DerivedTableAliasError, ParenthesesInConditionError, ValueListError


In [2]:
#schema file path
# fpath = '/home/22842219/Desktop/openSource/UnifiedSKGG-Cypher/data/text2cypher/schema.json'
fpath ='/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/schema.json'
db_id = 'concert_singer'


In [3]:
from process_cypher import Schema, get_schema_from_json
schema = Schema(get_schema_from_json(fpath, db_id))
schema.idMap

{'*': 0,
 '`concert_singer.stadium`.Name': 1,
 '`concert_singer.stadium`.Lowest': 2,
 '`concert_singer.stadium`.Stadium_ID': 3,
 '`concert_singer.stadium`.Capacity': 4,
 '`concert_singer.stadium`.Highest': 5,
 '`concert_singer.stadium`.Location': 6,
 '`concert_singer.stadium`.Average': 7,
 '`concert_singer.singer`.Country': 8,
 '`concert_singer.singer`.Age': 9,
 '`concert_singer.singer`.Name': 10,
 '`concert_singer.singer`.Song_Name': 11,
 '`concert_singer.singer`.Is_male': 12,
 '`concert_singer.singer`.Singer_ID': 13,
 '`concert_singer.singer`.Song_release_year': 14,
 '`concert_singer.concert`.Theme': 15,
 '`concert_singer.concert`.Stadium_ID': 16,
 '`concert_singer.concert`.concert_Name': 17,
 '`concert_singer.concert`.concert_ID': 18,
 '`concert_singer.concert`.Year': 19,
 '`concert_singer.stadium`': 0,
 '`concert_singer.singer`': 1,
 '`concert_singer.concert`': 2}

## debugging Cypher parser

In [4]:
from process_cypher import tokenize, scan_labels_with_alias

In [5]:
test_cypher = "MATCH (concert:`concert_singer.concert`)-[]-(T2:`concert_singer.stadium`)\nWITH T2.Name AS Name, count(*) AS count ORDER BY T2.Name \nRETURN Name,count"
                   #"match (n:`concert_singer.singer`) return count(*)"
toks = tokenize(test_cypher)
print(f'toks: {toks}')

+++++++++++++++++++++++++++++++++tokenize++++++++++++++++++++++++++++++
raw queries: ['MATCH (concert:`concert_singer.concert`)-[]-(T2:`concert_singer.stadium`)\nWITH T2.Name AS Name, count(*) AS count ORDER BY T2.Name \nRETURN Name,count'] <class 'list'>
toks: [('Token.Keyword', 'MATCH'), ('Token.Text.Whitespace', ' '), ('Token.Punctuation', '('), ('Token.Name.Variable', 'concert'), ('Token.Punctuation', ':'), ('Token.Name.Label', '`concert_singer.concert`'), ('Token.Punctuation', ')-['), ('Token.Punctuation', ']-('), ('Token.Name.Variable', 'T2'), ('Token.Punctuation', ':'), ('Token.Name.Label', '`concert_singer.stadium`'), ('Token.Punctuation', ')'), ('Token.Keyword', 'WITH'), ('Token.Text.Whitespace', ' '), ('Token.Name.Variable', 'T2'), ('Token.Operator', '.'), ('Token.Keyword', 'Name'), ('Token.Text.Whitespace', ' '), ('Token.Keyword', 'AS'), ('Token.Text.Whitespace', ' '), ('Token.Keyword', 'Name'), ('Token.Punctuation', ','), ('Token.Text.Whitespace', ' '), ('Token.Name.Functio

In [6]:
# test labels with alias
labels_with_alias = scan_labels_with_alias(toks)
print(labels_with_alias)

{'concert': '`concert_singer.concert`', 'T2': '`concert_singer.stadium`', 'Name': ':`concert_singer.stadium`.Name'}


In [7]:
start_idx = 0
# test parse_cypher
isBlock = False  #indicate if this is a block of cypher/subcypher
len_ = len(toks)
idx = start_idx
toks_ = [tok[1].lower() for tok in toks]

cypher = {}


In [8]:

from process_cypher import parse_cypher, parse_match, parse_with

In [9]:


print(f'idx: {idx}, toks_[idx]:{toks_[idx]}')
# parse from clause in order to get default tables
match_end_idx, table_units, default_tables = parse_match(
    toks, start_idx, labels_with_alias, schema
)
cypher['match']={'table_units': table_units}
print(cypher)

# parse 'with' clause
idx = match_end_idx
idx, with_units = parse_with(toks, idx, labels_with_alias, schema)
cypher['with'] = with_units


idx: 0, toks_[idx]:match
{'match': {'table_units': [('table_unit', 2), ('table_unit', 0)]}}


In [25]:
labels_with_alias[toks[34][1]]

'`concert_singer.stadium`'

In [ ]:
# test parse 'where' clause


In [26]:
def parse_col(toks, start_idx, labels_with_alias, schema, default_tables=None):
    """
        :returns next idx, column id
    """
    toks_ = [tok[1].lower() for tok in toks]
    tok = toks_[start_idx]
    if tok == "*":
        return start_idx + 1, schema.idMap[tok]
    print('ziiiiiii:', toks_[start_idx:])
    print(toks[start_idx:])
    if toks[start_idx][0] in ['Token.Punctuation', 'Token.Text.Whitespace' ]:
        start_idx+=1
    print('start_idx:', start_idx)
    if '.' in toks_[start_idx:] and toks[start_idx][0]=='Token.Name.Variable':  # if token is a composite
        print(tok)
        key = labels_with_alias[toks[start_idx][1]]
        schema_key = '{}.{}'.format(key, toks[start_idx+2][1])
        return start_idx+2, schema.idMap[schema_key]

    assert default_tables is not None and len(default_tables) > 0, "Default tables should not be None or empty"

    for alias in default_tables:
        table = labels_with_alias[alias]
        table_original = table.strip('`').split('.')[-1]
        if tok in schema.schema[table_original]:
            key = table + "." + tok
            return start_idx+1, schema.idMap[key]

    assert False, "Error col: {}".format(tok)


def parse_col_unit(toks, start_idx, labels_with_alias, schema, default_tables=None):
    """
        :returns next idx, (agg_op id, col_id)
    """
    toks_ = [tok[1].lower() for tok in toks]
    idx = start_idx
    len_ = len(toks)
    # isBlock = False
    isDistinct = False

    # if toks_[idx] == '(':
    #     isBlock = True
    #     idx += 1
    if toks[idx][0] in ['Token.Punctuation', 'Token.Text.Whitespace']:
        idx += 1

    if toks_[idx] in AGG_OPS:
        agg_id = AGG_OPS.index(toks_[idx])
        idx += 1
        assert idx < len_ and toks_[idx] == '('
        idx += 1
        if toks_[idx] == "distinct":
            idx += 1
            isDistinct = True
        idx, col_id = parse_col(toks, idx, labels_with_alias, schema, default_tables)
        assert idx < len_ and toks_[idx] == ')'
        idx += 1
        return idx, (agg_id, col_id, isDistinct)

    if toks_[idx] == "distinct":
        idx += 1
        isDistinct = True
    agg_id = AGG_OPS.index("none")
    idx, col_id = parse_col(toks, idx, labels_with_alias, schema, default_tables)

    # if isBlock:
    #     assert toks_[idx] == ')'
    #     idx += 1  # skip ')'

    return idx, (agg_id, col_id, isDistinct)

def parse_val_unit(toks, start_idx, labels_with_alias, schema, default_tables=None):
    idx = start_idx
    len_ = len(toks)
    # isBlock = False
    # if toks[idx] == '(':
    #     isBlock = True
    #     idx += 1

    col_unit1 = None
    col_unit2 = None
    unit_op = UNIT_OPS.index('none')

    idx, col_unit1 = parse_col_unit(toks, idx, labels_with_alias, schema, default_tables)
    if idx < len_ and toks[idx] in UNIT_OPS:
        unit_op = UNIT_OPS.index(toks[idx])
        idx += 1
        idx, col_unit2 = parse_col_unit(toks, idx, labels_with_alias, schema, default_tables)

    # if isBlock:
    #     assert toks[idx] == ')'
    #     idx += 1  # skip ')'

    return idx, (unit_op, col_unit1, col_unit2)


In [22]:
# test parse 'order by' clause
toks_ = [tok[1].lower() for tok in toks]
len_ = len(toks)
val_units = []  # (order_type, val_units), \
                    # where val_units = (unit_op, col_unit1, col_unit2), \
                    # and   col_unit=(agg_id, col_id, isDistinct), \
                    # and   col_id = schema.idMap[key]

order_type = "asc"  # default type is 'asc'

if idx>=len_ or toks_[idx]!='order by':
    print(idx, val_units)

idx+=1
print(toks[idx:])
while idx<len_ and not (toks_[idx] in CLAUSE_KEYWORDS or toks_[idx] in ( ";")):
    print(idx)
    idx, val_unit = parse_val_unit(toks, idx, labels_with_alias, schema, default_tables)
    val_units.append(val_unit)
    if idx < len_ and toks_[idx] in ORDER_OPS:
        order_type = toks_[idx]
        idx += 1
    if idx < len_ and toks[idx] == ',':
        idx += 1  # skip ','
    else:
        break




[('Token.Text.Whitespace', ' '), ('Token.Name.Variable', 'T2'), ('Token.Operator', '.'), ('Token.Keyword', 'Name'), ('Token.Text.Whitespace', ' \n'), ('Token.Keyword', 'RETURN'), ('Token.Text.Whitespace', ' '), ('Token.Keyword', 'Name'), ('Token.Punctuation', ','), ('Token.Name.Variable', 'count')]
33
ziiiiiii: ['t2', '.', 'name', ' \n', 'return', ' ', 'name', ',', 'count']
[('Token.Name.Variable', 'T2'), ('Token.Operator', '.'), ('Token.Keyword', 'Name'), ('Token.Text.Whitespace', ' \n'), ('Token.Keyword', 'RETURN'), ('Token.Text.Whitespace', ' '), ('Token.Keyword', 'Name'), ('Token.Punctuation', ','), ('Token.Name.Variable', 'count')]
start_idx: 34
t2
36 (0, 1, False)


AssertionError: 

In [ ]:
toks_[idx-1:]

In [ ]:
from process_cypher import parse_val_unit

In [ ]:
# test parse 'return' clause
# idx = start_idx
len_ = len(toks)
assert 'return' in toks_[idx:], "'return' not found"
idx += 1
isDistinct = False
if idx < len_ and toks_[idx] == 'distinct':
    idx += 1
    isDistinct = True
val_units = []
print(idx, toks[idx:])
while idx < len_ and toks_[idx] not in CLAUSE_KEYWORDS:
    agg_id = AGG_OPS.index("none")
    if toks[idx] in AGG_OPS:
        agg_id = AGG_OPS.index(toks[idx])
        idx += 1
    idx, val_unit = parse_val_unit(toks_, idx, labels_with_alias, schema, default_tables)

    val_units.append((agg_id, val_unit))
    if idx < len_ and toks_[idx] == ',':
        idx += 1  # skip ','


In [ ]:
gold0 = { "query": "MATCH (t1:`concert_singer.singer`)\n\
    WHERE t1.Song_Name =~'.*[H|h]ey.*'\nRETURN t1.Name,t1.Country",
        "question": "what is the name and nation of the singer who have a song having 'Hey' in its name?",
        "schema_path": "/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/schema.json",
        "db_id": "concert_singer",
        "db_tag_names": [
            ":`concert_singer.stadium`",
            ":`concert_singer.singer`",
            ":`concert_singer.concert`"
        ],
        "db_property_names": {
            "tag_id": [
                -1,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                1,
                1,
                1,
                1,
                1,
                1,
                1,
                2,
                2,
                2,
                2,
                2
            ],
            "property_name": [
                "*",
                "Name",
                "Lowest",
                "Stadium_ID",
                "Capacity",
                "Highest",
                "Location",
                "Average",
                "Country",
                "Age",
                "Name",
                "Song_Name",
                "Is_male",
                "Singer_ID",
                "Song_release_year",
                "Theme",
                "Stadium_ID",
                "concert_Name",
                "concert_ID",
                "Year"
            ]
        },
        "db_property_types": [
            "String",
            "Long",
            "Long",
            "Long",
            "Long",
            "String",
            "Long",
            "String",
            "Long",
            "String",
            "String",
            "String",
            "Long",
            "String",
            "String",
            "String",
            "String",
            "Long",
            "String"
        ],
        "serialized_schema": " | concert_singer | :`concert_singer.stadium` : Name , Lowest , Stadium_ID , Capacity , Highest , Location , Average | :`concert_singer.singer` : Country , Age , Name , Song_Name , Is_male , Singer_ID , Song_release_year | :`concert_singer.concert` : Theme , Stadium_ID , concert_Name , concert_ID , Year",
        "struct_in": "| concert_singer | :`concert_singer.stadium` : Name , Lowest , Stadium_ID , Capacity , Highest , Location , Average | :`concert_singer.singer` : Country , Age , Name , Song_Name , Is_male , Singer_ID , Song_release_year | :`concert_singer.concert` : Theme , Stadium_ID , concert_Name , concert_ID , Year",
        "text_in": "what is the name and nation of the singer who have a song having 'Hey' in its name?",
        "seq_out": "MATCH (singer:`concert_singer.singer`) WHERE singer.Song_Name =~'.*[H|h]ey.*' RETURN singer.Name,singer.Country",
        "description": "task: text-to-cypher",
        "section": "test",
        "arg_path": "META_TUNING/text2cypher_with_cell.cfg"
    }
pred0 = "MATCH (t1:`concert_singer.singer`) \
    WHERE t1.Song_Name =~'.*[Hey]?.*' RETURN t1.Name,t1.Nation"

In [ ]:
gold0['struct_in']

In [ ]:
toks0 = tokenize(pred0)
print(f'toks: {toks0}')


In [ ]:
gold1 = {'query': 'MATCH (singer:`concert_singer.singer`)\nWITH singer.Country AS Country, count(singer.Country) AS count\nRETURN Country,count', 'question': 'How many singers are from each country?', 'schema_path': '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/schema.json', 'db_id': 'concert_singer', 'db_tag_names': [':`concert_singer.stadium`', ':`concert_singer.singer`', ':`concert_singer.concert`'], 'db_property_names': {'tag_id': [-1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], 'property_name': ['*', 'Name', 'Lowest', 'Stadium_ID', 'Capacity', 'Highest', 'Location', 'Average', 'Country', 'Age', 'Name', 'Song_Name', 'Is_male', 'Singer_ID', 'Song_release_year', 'Theme', 'Stadium_ID', 'concert_Name', 'concert_ID', 'Year']}, 'db_property_types': ['String', 'Long', 'Long', 'Long', 'Long', 'String', 'Long', 'String', 'Long', 'String', 'String', 'String', 'Long', 'String', 'String', 'String', 'String', 'Long', 'String'], 'serialized_schema': ' | concert_singer | :`concert_singer.stadium` : Name , Lowest , Stadium_ID , Capacity , Highest , Location , Average | :`concert_singer.singer` : Country , Age , Name , Song_Name , Is_male , Singer_ID , Song_release_year | :`concert_singer.concert` : Theme , Stadium_ID , concert_Name , concert_ID , Year', 'struct_in': '| concert_singer | :`concert_singer.stadium` : Name , Lowest , Stadium_ID , Capacity , Highest , Location , Average | :`concert_singer.singer` : Country , Age , Name , Song_Name , Is_male , Singer_ID , Song_release_year | :`concert_singer.concert` : Theme , Stadium_ID , concert_Name , concert_ID , Year', 'text_in': 'How many singers are from each country?', 'seq_out': 'MATCH (singer:`concert_singer.singer`) WITH singer.Country AS Country, count(singer.Country) AS count RETURN Country,count', 'description': 'task: text-to-cypher', 'section': 'test', 'arg_path': 'META_TUNING/text2cypher_with_cell.cfg'}
pred1 = ' | concert_singer.singer| :`concert_singer.singer.stadium.singer.stadium.singer.stadium.singer.stadium.singer.singer.stadium.singer.singer.stadium.singer.singer.singer.stadium.singer.stadium.singer.singer.stadium.sing'

In [ ]:
gold2 = {'query': 'MATCH (tv_series:`tvshow.TV_series`)\nRETURN tv_series.Episode,tv_series.Rating\nORDER BY tv_series.Rating DESC\nLIMIT 3', 'question': "List top 3 highest Rating  TV series. List the TV series's Episode and Rating.", 'schema_path': '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/schema.json', 'db_id': 'tvshow', 'db_tag_names': [':`tvshow.TV_Channel`', ':`tvshow.TV_series`', ':`tvshow.Cartoon`', ':`tvshow.TV_Channel_HAS_tvshow.TV_series`'], 'db_property_names': {'tag_id': [-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2], 'property_name': ['*', 'Country', 'id', 'Package_Option', 'Language', 'Hight_definition_TV', 'Pixel_aspect_ratio_PAR', 'series_name', 'Pay_per_view_PPV', 'Content', 'id', 'Weekly_Rank', 'Episode', '18_49_Rating_Share', 'Rating', 'Share', 'Channel', 'Viewers_m', 'Air_Date', 'id', 'Title', 'Original_air_date', 'Production_code', 'Written_by', 'Directed_by', 'Channel']}, 'db_property_types': ['String', 'String', 'String', 'String', 'String', 'String', 'String', 'String', 'String', 'Double', 'Double', 'String', 'String', 'String', 'Double', 'String', 'String', 'String', 'Double', 'String', 'String', 'Double', 'String', 'String', 'String'], 'serialized_schema': ' | tvshow | :`tvshow.TV_Channel` : Country , id , Package_Option , Language , Hight_definition_TV , Pixel_aspect_ratio_PAR , series_name , Pay_per_view_PPV , Content | :`tvshow.TV_series` : id , Weekly_Rank , Episode , 18_49_Rating_Share , Rating , Share , Channel , Viewers_m , Air_Date | :`tvshow.Cartoon` : id , Title , Original_air_date , Production_code , Written_by , Directed_by , Channel | :`tvshow.TV_Channel_HAS_tvshow.TV_series` : ', 'struct_in': '| tvshow | :`tvshow.TV_Channel` : Country , id , Package_Option , Language , Hight_definition_TV , Pixel_aspect_ratio_PAR , series_name , Pay_per_view_PPV , Content | :`tvshow.TV_series` : id , Weekly_Rank , Episode , 18_49_Rating_Share , Rating , Share , Channel , Viewers_m , Air_Date | :`tvshow.Cartoon` : id , Title , Original_air_date , Production_code , Written_by , Directed_by , Channel | :`tvshow.TV_Channel_HAS_tvshow.TV_series` :', 'text_in': "List top 3 highest Rating  TV series. List the TV series's Episode and Rating.", 'seq_out': 'MATCH (tv_series:`tvshow.TV_series`) RETURN tv_series.Episode,tv_series.Rating ORDER BY tv_series.Rating DESC LIMIT 3', 'description': 'task: text-to-cypher', 'section': 'test', 'arg_path': 'META_TUNING/text2cypher_with_cell.cfg'}
pred2 = 'SELECT * FROMTV_Series_Has_TVshow.TV_Series,,,,,,,: `tvshow.TV_Series_Has_TVshow.TV_Series_Has_TVshow.TV_Series_Has_TVshow.TV_Series_Has_TVshow.TV_Series_Has_TVshow.TV_Series_Has_TVshow.TV_Series_Has_TV'
